<a href="https://colab.research.google.com/github/naveensaini/ConsoleHelloWorld/blob/master/trade_signal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
# !pip3 install auto-sklearn

import yfinance as yf
import ta
import pandas as pd
import numpy as np
import datetime
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
import autosklearn.regression
import matplotlib.pyplot as plt

BAR_DAYS = 7

# Download SPY data for first hour of trading for last 5 years
# start = datetime.datetime.today() - datetime.timedelta(days=365*5)
start = datetime.datetime.today() - datetime.timedelta(days=7)
end = datetime.datetime.today()
min_bars = yf.download('SPY', start=start, end=end, interval='1m')

# data = data.reset_index()

min_bars = min_bars.drop(columns=['Adj Close'])
# get data for first hour of trading
min_bars['DatetimeCol'] = min_bars.index

# min_bars = min_bars[pd.DatetimeIndex(min_bars[min_bars.DatetimeId]).hour <= 10]

# min_bars = min_bars[min_bars.index.to_series().dt.hour <= 10]
min_bars = min_bars[min_bars.index.hour <= 10]


min_bars

daily_bars = yf.download('SPY', start=start, end=end, interval='1d')
daily_bars = daily_bars.drop(columns=['Adj Close'])
daily_bars['DateCol'] = daily_bars.index

# daily_bars
# add a column for daily return
daily_bars['daily_return'] = daily_bars['Close'].pct_change() 
# daily_bars  
# add daily return for the next day to the min_bars
daily_bars['daily_return'] = daily_bars['daily_return'].shift(-1)


SMA_FAST_WINDOW = 8
SMA_SLOW_WINDOW = 21
MACD_FAST_WINDOW = 12
MACD_SLOW_WINDOW = 26
MFI_WINDOW = 14
STOCH_WINDOW = 14
RSI_WINDOW = 14
BB_WINDOW = 20

min_bars["ema_fast"] = ta.trend.ema_indicator(min_bars["Close"], window=SMA_FAST_WINDOW)
min_bars["ema_slow"] = ta.trend.ema_indicator(min_bars["Close"], window=SMA_SLOW_WINDOW)
# compare ema_fast and ema_slow for previous period
min_bars["ema_cross_up"] = np.where((min_bars["ema_fast"] > min_bars["ema_slow"]) & (min_bars["ema_fast"].shift(-1) < min_bars["ema_fast"].shift(-1)), 1, 0)
min_bars["ema_cross_down"] = np.where((min_bars["ema_fast"] < min_bars["ema_slow"]) & (min_bars["ema_fast"].shift(-1) > min_bars["ema_fast"].shift(-1)), 1, 0)

min_bars["macd"] = ta.trend.macd(min_bars["Close"], window_slow=MACD_SLOW_WINDOW, window_fast=MACD_FAST_WINDOW)
min_bars["macd_signal"] = ta.trend.macd_signal(min_bars["Close"], window_slow=MACD_SLOW_WINDOW, window_fast=MACD_FAST_WINDOW, window_sign=9)
# get macd cross up and down
min_bars["macd_cross_up"] = np.where((min_bars["macd"] > min_bars["macd_signal"]) & (min_bars["macd"].shift(-1) < min_bars["macd_signal"].shift(-1)), 1, 0)
min_bars["macd_cross_down"] = np.where((min_bars["macd"] < min_bars["macd_signal"]) & (min_bars["macd"].shift(-1) > min_bars["macd_signal"].shift(-1)), 1, 0)

min_bars["macd_diff"] = ta.trend.macd_diff(min_bars["Close"], window_slow=MACD_SLOW_WINDOW, window_fast=MACD_FAST_WINDOW, window_sign=9)

min_bars["macd_diff_bull"] = np.where((min_bars["macd_diff"].shift(-2) < 0) & (min_bars["macd_diff"].shift(-1) > min_bars["macd_diff"].shift(-2)) & (min_bars["macd_diff"] > min_bars["macd_diff"].shift(-1)), 1, 0)
min_bars["macd_diff_bear"] = np.where((min_bars["macd_diff"].shift(-2) > 0) & (min_bars["macd_diff"].shift(-1) < min_bars["macd_diff"].shift(-2)) & (min_bars["macd_diff"] < min_bars["macd_diff"].shift(-1)), 1, 0)

min_bars["mfi"] = ta.volume.money_flow_index(min_bars["High"], min_bars["Low"], min_bars["Close"], min_bars["Volume"], window=MFI_WINDOW)
min_bars["stoch"] = ta.momentum.stoch(min_bars["High"], min_bars["Low"], min_bars["Close"], window=STOCH_WINDOW)
min_bars["rsi"] = ta.momentum.rsi(min_bars["Close"], window=RSI_WINDOW)
min_bars["bb_high"] = ta.volatility.bollinger_hband(min_bars["Close"], window=BB_WINDOW)
min_bars["bb_low"] = ta.volatility.bollinger_lband(min_bars["Close"], window=BB_WINDOW)


# z= min_bars['DatetimeCol'].dt.date.map(daily_bars.set_index('DateCol')['daily_return'])
min_bars['return'] = min_bars['DatetimeCol'].dt.floor('d').map(daily_bars.set_index('DateCol')['daily_return'])
min_bars.drop(columns=['DatetimeCol', 'High', 'Low', 'Open', 'Close', 'Volume'], inplace=True)
# handle NaNs
# min_bars = min_bars.fillna(0)
# Remove records with NaNs
min_bars = min_bars.dropna()

# extract features and target
X = min_bars.drop(columns=['return'])
y = min_bars['return']
# min_bars
# daily_bars
# X.shape, y.shape
# do train test split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# X_train.shape, X_test.shape, y_train.shape, y_test.shape


# automl = autosklearn.regression.AutoSklearnRegressor()

automl = autosklearn.regression.AutoSklearnRegressor(time_left_for_this_task=60, per_run_time_limit=30)

# automl = autosklearn.regression.AutoSklearnRegressor(time_left_for_this_task=60, per_run_time_limit=30,tmp_folder='/temp/autosklearn_regression_example_tmp')

automl.fit(X_train, y_train)

from sklearn.metrics import mean_absolute_error
from autosklearn.metrics import mean_absolute_error, auto_mean_absolute_error

# summarize the results of the grid search
print(automl.sprint_statistics())

ModuleNotFoundError: ignored

In [23]:
!pip install auto-sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 46.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 57.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.9/260.9 KB 25.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 91.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 91.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 KB 5.3 MB/s eta 0:00:00
  Created wheel for auto-sklearn: filename=auto_sklearn-0.15.0-py3-none-any.whl size=6641945 sha256=d6215f39f6fdf17a46501ae0d0a39f5fd75687fb5899e326e4cebfe7e1489f91

In [18]:
import yfinance as yf
import ta
import pandas as pd
import numpy as np
import datetime
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
# import autosklearn.regression
import matplotlib.pyplot as plt

BAR_DAYS = 7

# Download SPY data for first hour of trading for last 5 years
# start = datetime.datetime.today() - datetime.timedelta(days=365*5)
start = datetime.datetime.today() - datetime.timedelta(days=7)
end = datetime.datetime.today()
min_bars = yf.download('SPY', start=start, end=end, interval='1m')

# data = data.reset_index()

min_bars = min_bars.drop(columns=['Adj Close'])
# get data for first hour of trading
min_bars['DatetimeId'] = min_bars.index

# min_bars = min_bars[pd.DatetimeIndex(min_bars[min_bars.DatetimeId]).hour <= 10]

# min_bars = min_bars[min_bars.index.to_series().dt.hour <= 10]
min_bars = min_bars[min_bars.index.hour <= 10]


min_bars

daily_bars = yf.download('SPY', start=start, end=end, interval='1d')
daily_bars = daily_bars.drop(columns=['Adj Close'])
daily_bars['DateId'] = daily_bars.index
# daily_bars
# add a column for daily return
daily_bars['daily_return'] = daily_bars['Close'].pct_change() 
# daily_bars  
# add daily return for the next day to the min_bars
daily_bars['daily_return'] = daily_bars['daily_return'].shift(-1)


SMA_FAST_WINDOW = 8
SMA_SLOW_WINDOW = 21
MACD_FAST_WINDOW = 12
MACD_SLOW_WINDOW = 26
MFI_WINDOW = 14
STOCH_WINDOW = 14
RSI_WINDOW = 14
BB_WINDOW = 20

min_bars["ema_fast"] = ta.trend.ema_indicator(min_bars["Close"], window=SMA_FAST_WINDOW)
min_bars["ema_slow"] = ta.trend.ema_indicator(min_bars["Close"], window=SMA_SLOW_WINDOW)
# compare ema_fast and ema_slow for previous period
min_bars["ema_cross_up"] = np.where((min_bars["ema_fast"] > min_bars["ema_slow"]) & (min_bars["ema_fast"].shift(-1) < min_bars["ema_fast"].shift(-1)), 1, 0)
min_bars["ema_cross_down"] = np.where((min_bars["ema_fast"] < min_bars["ema_slow"]) & (min_bars["ema_fast"].shift(-1) > min_bars["ema_fast"].shift(-1)), 1, 0)

min_bars["macd"] = ta.trend.macd(min_bars["Close"], window_slow=MACD_SLOW_WINDOW, window_fast=MACD_FAST_WINDOW)
min_bars["macd_signal"] = ta.trend.macd_signal(min_bars["Close"], window_slow=MACD_SLOW_WINDOW, window_fast=MACD_FAST_WINDOW, window_sign=9)
# get macd cross up and down
min_bars["macd_cross_up"] = np.where((min_bars["macd"] > min_bars["macd_signal"]) & (min_bars["macd"].shift(-1) < min_bars["macd_signal"].shift(-1)), 1, 0)
min_bars["macd_cross_down"] = np.where((min_bars["macd"] < min_bars["macd_signal"]) & (min_bars["macd"].shift(-1) > min_bars["macd_signal"].shift(-1)), 1, 0)

min_bars["macd_diff"] = ta.trend.macd_diff(min_bars["Close"], window_slow=MACD_SLOW_WINDOW, window_fast=MACD_FAST_WINDOW, window_sign=9)

min_bars["macd_diff_bull"] = np.where((min_bars["macd_diff"].shift(-2) < 0) & (min_bars["macd_diff"].shift(-1) > min_bars["macd_diff"].shift(-2)) & (min_bars["macd_diff"] > min_bars["macd_diff"].shift(-1)), 1, 0)
min_bars["macd_diff_bear"] = np.where((min_bars["macd_diff"].shift(-2) > 0) & (min_bars["macd_diff"].shift(-1) < min_bars["macd_diff"].shift(-2)) & (min_bars["macd_diff"] < min_bars["macd_diff"].shift(-1)), 1, 0)

min_bars["mfi"] = ta.volume.money_flow_index(min_bars["High"], min_bars["Low"], min_bars["Close"], min_bars["Volume"], window=MFI_WINDOW)
min_bars["stoch"] = ta.momentum.stoch(min_bars["High"], min_bars["Low"], min_bars["Close"], window=STOCH_WINDOW)
min_bars["rsi"] = ta.momentum.rsi(min_bars["Close"], window=RSI_WINDOW)
min_bars["bb_high"] = ta.volatility.bollinger_hband(min_bars["Close"], window=BB_WINDOW)
min_bars["bb_low"] = ta.volatility.bollinger_lband(min_bars["Close"], window=BB_WINDOW)


# min_bars["return"] = min_bars["DatetimeId"].dt.date.map(daily_bars.set_index("DateId")["daily_return"])
min_bars['new'] = min_bars['DatetimeId'].dt.floor('d').map(daily_bars.set_index('DateId')['daily_return'])

min_bars.drop(columns=['DatetimeId', 'High', 'Low', 'Open', 'Close', 'Volume'], inplace=True)
# daily_bars
min_bars

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,ema_fast,ema_slow,ema_cross_up,ema_cross_down,macd,macd_signal,macd_cross_up,macd_cross_down,macd_diff,macd_diff_bull,macd_diff_bear,mfi,stoch,rsi,bb_high,bb_low,new
Datetime,,,,,,,,,,,,,,,,,
2023-01-18 09:30:00-05:00,NaN,NaN,0,0,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,-0.00728
2023-01-18 09:31:00-05:00,NaN,NaN,0,0,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,-0.00728
2023-01-18 09:32:00-05:00,NaN,NaN,0,0,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,-0.00728
2023-01-18 09:33:00-05:00,NaN,NaN,0,0,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,-0.00728
2023-01-18 09:34:00-05:00,NaN,NaN,0,0,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,-0.00728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-24 10:55:00-05:00,399.951809,399.628792,0,0,0.276997,0.208552,0,0,0.068445,0,1,89.478880,94.695040,69.668115,400.474386,398.500052,NaN
2023-01-24 10:56:00-05:00,400.078072,399.709810,0,0,0.311738,0.229189,0,0,0.082549,0,0,89.715903,100.000000,71.599929,400.588369,398.572068,NaN
2023-01-24 10:57:00-05:00,400.186278,399.787555,0,0,0.338995,0.251150,0,0,0.087844,0,0,90.257458,85.985111,72.192072,400.676265,398.675673,NaN


In [ ]:
!pip inst

In [2]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 KB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 KB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 KB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 48.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1
  Attempting uninstall: html5lib
    Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
   

In [3]:
!pip install ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.10.2-py3-none-any.whl size=29106 sha256=35425cfe800be25d43184f6ee011dec8a58bc427c909df25d38e3d521c2a6b24
  Stored in directory: /root/.cache/pip/wheels/1b/81/50/db21d9643326c61dd6fee1dc2562d8351ee7753164c24ead38
Successfully built ta
